In [101]:
import pandas as pd
import gc

In [102]:
# Чтение файла fines.csv
fines = pd.read_csv('../data/data_df/fines.csv')

In [103]:
%%timeit  -r 10 -n 10
# Функция, которая проходит по фрейму данных через range(0, len(fines))
def loop(fines):
    result = []
    for i in range(0, len(fines)):
        result.append(fines.iloc[i]['Fines'] / fines.iloc[i]['Refund'] * fines.iloc[i]['Year'])
    return result
fines['FinesPerYear'] = loop(fines)

48.3 ms ± 3.14 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [104]:
%%timeit  -r 10 -n 10
# Функция, которая проходит по фрейму данных через fines.iterrows()
def iterr(fines):
    result = []
    for index, _ in fines.iterrows():
        result.append(fines.iloc[index]['Fines'] / fines.iloc[index]['Refund'] * fines.iloc[index]['Year'])
    return result
fines['FinesPerYear'] = iterr(fines)

74.2 ms ± 3.26 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [105]:
%%timeit  -r 10 -n 10
# Функция, которая проходит по фрейму данных через apply()
fines['FinesPerYear'] = fines.apply(lambda fines: fines['Fines'] / fines['Refund'] * fines['Year'], axis=1)

4.03 ms ± 59.2 μs per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [106]:
%%timeit  -r 10 -n 10
# Функция, которая проходит по фрейму данных через объект Series
fines['FinesPerYear'] = fines['Fines'] / fines['Refund'] * fines['Year']

119 μs ± 19.5 μs per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [107]:
%%timeit  -r 10 -n 10
# Функция, которая проходит по фрейму данных через метод .values
fines['FinesPerYear'] = fines['Fines'].values / fines['Refund'].values * fines['Year'].values

48.1 μs ± 9.57 μs per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [108]:
%%timeit  -r 10 -n 10
# Получение строки для определенного номера автомобиля
line = fines[fines['CarNumber'] == '9763HY161RUS']

150 μs ± 14.7 μs per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [109]:
# Установка индекса CarNumber
fines = fines.set_index('CarNumber')

In [110]:
%%timeit  -r 10 -n 10
# Получение строки для определенного номера автомобиля через индекс
line = fines[fines.index == '9763HY161RUS']

120 μs ± 35.6 μs per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [111]:
# Использование памяти фреймом fines
fines.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 930 entries, Y163O8161RUS to К456DE77
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Refund        930 non-null    int64  
 1   Fines         930 non-null    float64
 2   Make          930 non-null    object 
 3   Model         930 non-null    object 
 4   Year          930 non-null    int64  
 5   FinesPerYear  930 non-null    float64
dtypes: float64(2), int64(2), object(2)
memory usage: 204.3 KB


In [112]:
# Оптимизация памяти в фрейме fines
for col in fines.columns:
    col_type = fines[col].dtype
    if col_type == 'object':
        fines[col] = fines[col].astype('category')
    elif col_type in ['int64', 'int32', 'int16', 'int8']:
        fines[col] = pd.to_numeric(fines[col], downcast='integer')
    elif col_type in ['float64', 'float32']:
        fines[col] = pd.to_numeric(fines[col], downcast='float')
fines.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 930 entries, Y163O8161RUS to К456DE77
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   Refund        930 non-null    int8    
 1   Fines         930 non-null    float32 
 2   Make          930 non-null    category
 3   Model         930 non-null    category
 4   Year          930 non-null    int16   
 5   FinesPerYear  930 non-null    float64 
dtypes: category(2), float32(1), float64(1), int16(1), int8(1)
memory usage: 80.6 KB
